https://bitcoin.stackexchange.com/questions/12427/can-someone-explain-how-the-bitcoin-blockchain-works/42515#42515

The Bitcoin system orders transactions by placing them in groups
called blocks and then linking these blocks through what is called Blockchain. 
`The transactions in one block are considered to have happened at the same time.`

The fingerprint, called a hash, takes a lot of computing time and energy to generate initially(This time and energy increases or decreases depending upon the total computing power present with the miners). It thus serves as a proof that the miner who added the block to the blockchain did the computational work to earn a bitcoin reward (for this reason, Bitcoin is said to use a “proof-of-work” protocol). This hash is just a merkle hash of all the transactions included in this block. It also serves as a kind of seal, since altering the block would require generating a new hash. Verifying whether or not the hash matches its block, however, is easy, and once the nodes have done so they update their respective copies of the blockchain with the new block. This is the consensus protocol.

The final security element is that the hashes also serve as the links in the blockchain: each block includes the previous block’s unique hash. So if you want to change an entry in the ledger retroactively, you have to calculate a new hash not only for the block it’s in but also for every subsequent block. And you have to do this faster than the other nodes can add new blocks to the chain (On an average a new block is added every 10 minutes). So unless you have computers that are more powerful than the rest of the nodes combined (and even then, success isn’t guaranteed), all the alterations needs to be done in 10 minutes window, any blocks you add will conflict with existing ones, and the other nodes will automatically reject your alterations. This is what makes the blockchain tamperproof, or “immutable.”

__proof-of-work algorithm:__ 

each miner receives a list of pending transactions from the network,
the miner takes a certain number of transactions(greater than a threshold) and attempts to generate a block by finding an answer to  a mathematical problem.
Example mathematical problem: find a number that when combined with transaction data, previous block's hash and the current timestamp generates a hash containing a certain x number of leading zeroes.
It involves a lot of computation to find the answer.

But say the answer is 886, now verifying whether the number 886 along with all the other data generates a hash of x leading zeroes is very easy.
The other nodes (non-miners as well) then do the verification task and append the block to their chain. Then obiously if they were mining, they have to mine with a new value of previous hash.
The task of verification and updation of blockchain on the node is handled based on `consensus protocol`.

code from : https://gist.github.com/aunyks/47d157f8bc7d1829a729c2a6a919c173

my thoughts on additions to this code:
 - each transaction must involve a private key signature from the sender to prevent others creating false transaction

 - consensus algorithm should invoolve making sure that the other chains received have valid blocks, basically instead of getting all other chains, just get the newly generated block.This function is still a confusion for me.

 - poW algo obviously involves a problem like the one given above and not just incrementing to a multiple of 9

doubts:
https://bitcoin.stackexchange.com/questions/95504/do-miners-validate-each-others-blocks/95509#95509

https://bitcoin.stackexchange.com/questions/61833/what-is-the-exact-moment-when-a-miner-gets-rewarded

https://bitcoin.stackexchange.com/questions/57683/does-the-block-reward-comes-from-a-bitcoin-treasury-if-yes-then-how-is-it-issu

In [50]:
from flask import Flask #flask is similar to django
from flask import request
import json
import requests
import hashlib as hasher
import datetime as date

In [51]:
node = Flask(__name__)

we are instantiating a Flask object by passing `__name__` argument to the Flask constructor. The Flask constructor has one required argument which is the name of the application package. Most of the time `__name__` is the correct value.

In [52]:
class Block:
    def __init__(self,index,timestamp,data,previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash #should this be stored? security?
        self.hash = self.hash_block() #create current hash
        
    def hash_block(self):
        sha = hasher.sha256()
        #masala to hash upon, note: it needs to be encoded to utf-8
        masala = (str(self.index)+str(self.timestamp)+str(self.data)+str(self.previous_hash)).encode('utf-8') 
        sha.update(masala)
        return sha.hexdigest()

In [53]:
#create first block i.e genesis block

def create_genesis_block():
    initial_data = {
        "proof-of-work": 9,
        "transactions" : None
    }
    return Block(0,date.datetime.now(), initial_data , "NoPreviousHashYet")

In [54]:
# A completely random address of the owner of this node
miner_address = "q3nf394hjg-random-miner-address-34nf3i4nflkn3oi"

In [55]:
blockchain = []
blockchain.append(create_genesis_block())

#store transaction of this node as json objects
mynode_transactions = []


#store url of peer nodes
peer_nodes = []

# a variable to decide if we are mining or not
mining = True

In [56]:
@node.route('/txion', methods=['POST'])
def transaction():
    #a json object is sent here containing details of the new transaction
    new_transaction = request.get_json()
    mynode_transactions.append(new_transaction)
    
    print("New transaction")
    print("From:",new_transaction["from"].encode('ascii','replace'))
    print("To:",new_transaction["to"].encode('ascii','replace'))
    print("Amount:",new_transaction["amount"])
    
    #now send the client who sent request a msg
    return "Transaction successful\n"

`Do not execute above function again` or you will get assertion error

/txion has already been mapped to transaction().
Executing it again will make it think that you are assigning /txion to another endpoint which btw is also transaction()

In [57]:
@node.route('/getchain', methods=['GET'])
def send_chain():
    
    chain_to_send = []
    
    #convert blocks into dictionaries so that they can be sent as json objects
    for block in blockchain:
        chain_to_send.append({
            "index": block.index,
            "timestamp": block.timestamp,
            "data" : block.data,
            "hash" : block.hash
        })
        
    #convert the list of dicts into json object
    chain_to_send = json.dumps(chain_to_send)
    return  chain_to_send

In [ ]:
def get_peer_chains():
    #get blockchains of peer nodes
    
    other_chains  = []
    for node_url in peer_nodes:
        
        #get their chains using a get request
        chain = requests.get(node_url+"/blocks").content
        
        #convert JSON object to a python dictionary
        chain = json.loads(block)
        
        #Add it to our list
        other_chains.append(chain)
    return other_chains

In [ ]:
def consensus():
    other_chains = get_peer_chains()
    
    #we are going store longest chain as valid chain
    longest_chain = blockchain
    for chain in other_chains:
        if(len(longest_chain)) < len(chain):
            longest_chain = chain
    
    blockchain = longest_chain

In [ ]:
def proof_of_work(last_proof):
    # Create a variable that we will use to find
    # our next proof of work
    incrementor = last_proof + 1
    
    #we have taken mining task to be to find next multiple of last_proof
    while not (incrementor % 9 == 0 and incrementor % last_proof == 0):
        incrementor += 1
    # Once that number is found,
    # we can return it as a proof
    # of our work
    return incrementor

In [ ]:
@node.route('/mine', methods = ['GET'])
def mine():
    
    last_block = blockchain[-1]
    last_proof = last_block.data['proof-of-work']
    
    # mine for block, and get proof of work
    proof = proof_of_work(last_proof)
    
    mynode_transactions.append(
    { "from": "network", "to": miner_address, "amount": 1 }
    )
    
    new_block_data = {
        "proof-of-work": proof,
        "transactions": list(mynode_transactions)
    }
    new_block_index = last_block.index + 1
    new_block_timestamp = this_timestamp = date.datetime.now()
    last_block_hash = last_block.hash
    #empty transaction list as all transactions have been appended to  new_block_data
    mynode_transactions[:] = []
    #now create new block
    
    mined_block = Block(
    new_block_index,
    new_block_timestamp,
    new_block_data,
    last_block_hash
    )
    blockchain.append(mined_block)
    
    #let client (shouldn't it be all peers?) know that we mined a block
    return json.dumps({
        "index" : new_block_index,
        "timestamp" : str(new_block_timestamp),
        "data" : new_block_data,
        "hash" : last_block_hash
    }) + "\n"

node.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Apr/2020 11:22:12] "POST /txion HTTP/1.1" 200 -


New transaction
From: b'akjflw'
To: b'fjlakdj'
Amount: 3


127.0.0.1 - - [26/Apr/2020 11:22:14] "GET /mine HTTP/1.1" 200 -
